In [1]:
import numpy
import ipywidgets as widgets 
from ipywidgets import interact, interactive_output, interactive 
import yaml 

In [2]:
    
with open('dataviz_lab.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    lab_dict = yaml.load(file, Loader=yaml.FullLoader)

    print(lab_dict)
    

{'q0': {'question': 'What are the three components of a data story?', 'options': {'a': 'Questions, Answers, Visualizations', 'b': 'EDA, Modelling and Visualizations', 'c': 'Questions, Data, and Visualizations'}, 'question_type': '0', 'answer': 'Questions, Data, and Visualizations'}, 'q1': {'question': 'what pandas command would you use to view the first 10 entries in a dataframe?', 'options': {'a': 'my_dataframe.tail()', 'b': 'my_dataframe.head(10)', 'c': 'my_dataframe.head(20)'}, 'question_type': '0', 'answer': 'my_dataframe.head(10)'}, 'q2': {'question': 'How much did Europe contribute to the World GDP in the 2007?', 'options': {'a': '33.4%', 'b': '25.5%', 'c': '35.6%', 'd': '1.0%'}, 'question_type': '0', 'answer': '25.5%'}, 'q3': {'question': 'What was the life expectancy of Swiss person in the year 1962 according to the animated plot. Pausing the animated plot may help?', 'options': {'a': '71.32 years', 'b': '69.45 years', 'c': '44.50 years'}, 'question_type': '0', 'answer': '71.32

In [3]:

def qtype1(options, qnum): 
    check_dict = {}

    for x in options.keys():
            key_name = str(qnum)+"_"+str(x)
            check_dict[key_name] = widgets.Checkbox(value=False,
                                             description=options[x],
                                             disabled=False,
                                             indent=False)    
    return check_dict

def qtype2(single_options, qnum2):
    output_widget = {}
    key_name = str(qnum2)+"_0"
    output_widget[key_name] = widgets.RadioButtons(
                                    options=list(single_options.values()),
                                #    value='pineapple', # Defaults to 'pineapple'
                                   layout={'width': 'max-content'}, # If the items' names are long
                                    description= " ",
                                    disabled=False
                                )

    return output_widget

        
def options_widgets(options, qnum, qtype): 
    
    all_ouput_widgets = {}
    if qtype == 0: 
        output_widget = qtype2(options, qnum)
        all_ouput_widgets.update(output_widget)
    elif qtype ==1: 
        output_widgets = qtype1(options, qnum)
        all_ouput_widgets.update(output_widgets)
    return all_ouput_widgets


def question_widget(question, select_crit): 
    question_string = "{}  ({})".format(question, select_crit)
    question_put = f"<b><font size='2'>{question_string}</b>"
    return widgets.HTML(value= question_put)

qkeys = list(lab_dict.keys())



all_widgets = []
new_widgets = {}
all_answers = {}
for x in range(0, len(lab_dict)): 
        
        single_question = lab_dict[qkeys[x]]
        
        question = single_question["question"]
        options = single_question["options"]
        qtype = int(single_question["question_type"])
        all_answers[x] = single_question["answer"]
        opt_widgets = options_widgets(options, x, qtype)
        old_widgets = opt_widgets
        new_widgets.update(old_widgets)
        
        
        qwidget = question_widget(question, qtype)
        all_widgets.append(qwidget)
        
        list_widgets = list(old_widgets.values())
        for x in list_widgets: 
            all_widgets.append(x)
        

# adding submit button 
submit_button = widgets.Button(description="Submit")
all_widgets.append(submit_button)


In [4]:
all_answers

{0: 'Questions, Data, and Visualizations',
 1: 'my_dataframe.head(10)',
 2: '25.5%',
 3: '71.32 years',
 4: ['Stories are extremely effective when told using appropriate data and intuitive visualizations.',
  'Choosing the right visualization and selecting and filtering the right data for the visualization, are both crucial.',
  'visualizations should use lots of colors to make an impact'],
 5: ['Stories are extremely effective when told using appropriate data and intuitive visualizations.']}

In [5]:
def store_answers(all_answers, new_widgets): 
    # get output from the widgets 
    
    store_data = {}
    type_to_value = {list: [], float: 0.0, int: 0, str: 0}

    # initalize store data
    for qnum in all_answers.keys(): 
        ans_type = type(all_answers[qnum])
        store_data[qnum] = type_to_value[ans_type]

    for key in new_widgets:             
        key_split = key.split("_")
        q_number = int(key_split[0])

        if key_split[1] == "0": 
            answer_value = key_split[0] +"_"+str(new_widgets[key].index) 
            store_data[q_number] = new_widgets[key].value


        if new_widgets[key].value is True: 

            answered = new_widgets[key].description
            temp = store_data[q_number].copy()
            temp.append(answered)
            store_data[q_number] = temp 

    return store_data

def grade_answers(store_data, all_answers ):
    questions_answered = []
    score_assigned = {}
    type_to_value = {list: [], float: 0.0, int: 0, str: 0}

    # initalize score_assigned 
    for qnum in all_answers.keys(): 
        ans_type = type(all_answers[qnum])
        score_assigned[qnum] = type_to_value[ans_type]

    for q_num in all_answers.keys(): 
        if q_num in list(store_data.keys()): 
            questions_answered.append(q_num)

            # based on type of question do the appropriate comparison

            # type 0 is just single answer mcq 
            qkey = "q"+str(q_num)
            if lab_dict[qkey]["question_type"] == "0": 
                if store_data[q_num] == all_answers[q_num]: 
                        score_assigned[q_num] = 1.0
                else: 
                        score_assigned[q_num] = -0.25

            # type 1 which mean has multiple answers
            if lab_dict[qkey]["question_type"] == "1": 
                
                # if answer is in all_answers then give a score of 1
                # else score of -0.25
                for ans in all_answers[q_num]: 
                        if ans in score_assigned[q_num]: 
                            temp = score_assigned[q_num].copy()
                            temp.append(1.0)
                            score_assigned[q_num] = temp 
                        else: 
                            temp = score_assigned[q_num].copy()
                            temp.append(-0.25)
                            score_assigned[q_num] = temp 

             

    return score_assigned

In [6]:


ui = widgets.VBox(all_widgets)


def f(**kwargs):

    return None

output = widgets.interactive_output(f, new_widgets)
display(ui, output)


indicies = []
store_data = {}
questions_answered = []
score_assigned = {}
def on_button_clicked(b):
    
    with output:
        store_data = store_answers(all_answers, new_widgets)
        score_assigned = grade_answers(store_data, all_answers)
        
    #         # get answers and store them based on question number
#        # get output from the widgets 
      
#         type_to_value = {list: [], float: 0.0, int: 0.0, str: 0.0}

#         # initalize store data
#         for qnum in all_answers.keys(): 
#             ans_type = type(all_answers[qnum])
#             store_data[qnum] = type_to_value[ans_type]

#         for key in new_widgets:             
#             key_split = key.split("_")
#             q_number = int(key_split[0])

#             if key_split[1] == "0": 
#                 answer_value = key_split[0] +"_"+str(new_widgets[key].index) 
#                 store_data[q_number] = new_widgets[key].value


#             if new_widgets[key].value is True: 

#                     answered = new_widgets[key].description
#                     store_data[q_number].append(answered)

#         # grading 
       
        


#         # initalize score_assigned 
#         for qnum in all_answers.keys(): 
#             ans_type = type(all_answers[qnum])
#             score_assigned[qnum] = type_to_value[ans_type]

#         for q_num in all_answers.keys(): 
#             if q_num in list(store_data.keys()): 
#                 questions_answered.append(q_num)

#                 # based on type of question do the appropriate comparison

#                 # type 0 is just single answer mcq 
#                 qkey = "q"+str(q_num)
                
#                 if lab_dict[qkey]["question_type"] == "0": 
#                     if store_data[q_num] == all_answers[q_num]: 
#                             score_assigned[q_num] = 1.0
#                     else: 
#                             score_assigned[q_num] = -0.25

#                 # type 1 which mean has multiple answers
#                 if lab_dict[qkey]["question_type"] == "1": 
                   
#                     # if answer is in all_answers then give a score of 1
#                     # else score of -0.25
#                     for ans in store_data[q_num]: 
#                         print(ans)
#                         if ans in all_answers[q_num]: 
#                             score_assigned[q_num].append(1.0)
#                         else: 
#                             score_assigned[q_num].append(-0.25)
                        



                
        print("your Scores:", score_assigned)
submit_button.on_click(on_button_clicked)



Output()

## Grading Module


### Dict with student answers with question number as key


In [7]:
def store_answers(all_answers, new_widgets): 
#     get output from the widgets 

    store_data = {}
    type_to_value = {list: [], float: 0.0, int: 0, str: 0}

    # initalize store data
    for qnum in all_answers.keys(): 
        ans_type = type(all_answers[qnum])
        store_data.setdefault(qnum, type_to_value[ans_type])

    for key in new_widgets:             
        key_split = key.split("_")
        q_number = int(key_split[0])

        if key_split[1] == "0": 
            answer_value = key_split[0] +"_"+str(new_widgets[key].index) 
            store_data[q_number] = new_widgets[key].value

        if new_widgets[key].value is True: 

            answered = new_widgets[key].description
            temp = store_data[q_number].copy()
            temp.append(answered)
            store_data[q_number] = temp 

      
    return store_data

store_data = store_answers(all_answers, new_widgets)

In [8]:
store_data

{0: 'Questions, Answers, Visualizations',
 1: 'my_dataframe.tail()',
 2: '33.4%',
 3: '71.32 years',
 4: [],
 5: []}

### Dict of answer key

In [9]:
all_answers

{0: 'Questions, Data, and Visualizations',
 1: 'my_dataframe.head(10)',
 2: '25.5%',
 3: '71.32 years',
 4: ['Stories are extremely effective when told using appropriate data and intuitive visualizations.',
  'Choosing the right visualization and selecting and filtering the right data for the visualization, are both crucial.',
  'visualizations should use lots of colors to make an impact'],
 5: ['Stories are extremely effective when told using appropriate data and intuitive visualizations.']}

### comparing both answer key and student answers 


In [10]:
def grade_answers(store_data, all_answers ):
    questions_answered = []
    score_assigned = {}
    type_to_value = {list: [], float: 0.0, int: 0, str: 0}

    # initalize score_assigned 
    for qnum in all_answers.keys(): 
            ans_type = type(all_answers[qnum])
            score_assigned.setdefault(qnum, type_to_value[ans_type])
    
    for q_num in all_answers.keys(): 
        if q_num in list(store_data.keys()): 
            questions_answered.append(q_num)

            # based on type of question do the appropriate comparison

            # type 0 is just single answer mcq 
            qkey = "q"+str(q_num)
            if lab_dict[qkey]["question_type"] == "0": 
                if store_data[q_num] == all_answers[q_num]: 
                        score_assigned[q_num] = 1.0
                else: 
                        score_assigned[q_num] = -0.25

            # type 1 which mean has multiple answers
            if lab_dict[qkey]["question_type"] == "1": 
               
                # if answer is in all_answers then give a score of 1
                # else score of -0.25
                for ans in all_answers[q_num]: 
                    if ans in store_data[q_num]: 
                        temp = score_assigned[q_num].copy()
                        temp.append(1.0)
                        score_assigned[q_num] = temp 
                    else: 
                        temp = score_assigned[q_num].copy()
                        temp.append(-0.25)
                        score_assigned[q_num] = temp 
                
             
                    
    return score_assigned


score_assigned = grade_answers(store_data, all_answers)
        

In [11]:
score_assigned

{0: -0.25, 1: -0.25, 2: -0.25, 3: 1.0, 4: [-0.25, -0.25, -0.25], 5: [-0.25]}

### Calculate final score 

In [12]:
score_calcuated = 0 
for key in score_assigned.keys(): 
    if type(score_assigned[key]) is float: 
        score_calcuated += score_assigned[key]
    if type(score_assigned[key]) is list: 
        score_calcuated += sum(score_assigned[key])/len(all_answers[key])
        
print(score_calcuated)


-0.25


In [13]:
len(all_answers[key])

1

In [14]:
# grading module. Flattening answers 

all_answers_flattened = []
for x in all_answers:
    if type(x) is list: 
        for y in x: 
            all_answers_flattened.append(y)
    else: 
        all_answers_flattened.append(x)
    

In [15]:
qkeys = list(new_widgets.keys())
qcount =  [int(x.split("_")[0]) for x in qkeys ]

unique_list = []
results_dict = {}

for counter, x in enumerate(qcount): 
    if x not in unique_list: 
        results_dict[x] = qkeys[counter].split("_")[1]
        unique_list.append(x)
       
    else: 
        if type(results_dict[x]) is not list: 
            results_dict[x] = [results_dict[x]]
            results_dict[x].append(qkeys[counter].split("_")[1])
        else: 
            results_dict[x].append(qkeys[counter].split("_")[1])
        